In [ ]:
!gdown 1dilkpbKOLgFHMxVZLR8CJkRNeN7bS4EB #Download_part

In [ ]:
#Adding libraries
!pip install folium
!pip install jdatetime

In [ ]:
#IMPORTS_PART

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
from jdatetime import datetime as jdatetime
import datetime


In [ ]:
Divar_data = pd.read_csv("/content/Divar.csv")


توزیع آگهی‌های موجود در دسته‌های مختلف را برای دسته‌بندی سطح دو و سطح سه رسم کنید.

In [ ]:
chosen1 = Divar_data[["cat2_slug"]]

chosen1_value_counts = chosen1.value_counts()
chosen1_value_counts

In [ ]:
chosen1_value_counts.plot(kind = 'pie')

In [ ]:
chosen2 = Divar_data[["cat3_slug"]]

chosen2_value_counts = chosen2.value_counts()

chosen2_value_counts.plot.bar(x = 'lables', y = 'count')

هیستوگرام سال ساخت را رسم کنید.

In [ ]:
def timestamptomonth(series):
    l_ist = []
    for item in series:
        l_ist.append(item.month_name())
    return pd.Series(l_ist)

def timestamptoyear(dataframe):
    l_ist = []
    for item in dataframe:
        l_ist.append(item.year)
    return pd.Series(l_ist)



In [ ]:
time = pd.to_datetime(Divar_data["created_at_month"])
timestmap = timestamptomonth(time)
year = timestamptoyear(time)
yeardistro = pd.DataFrame({'years': [2024,2023,2025,2022,2021,2020], 'counts': year.value_counts()})
plt.figure(figsize=(10, 6))
sns.histplot(year, bins=10,kde=True, color='skyblue', edgecolor='black')

plt.title('tozie sal sakht', fontsize=14)
plt.xlabel('salsakht', fontsize=12)
plt.ylabel( 'tedadmelk', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.xticks(sorted(year.unique()))


plt.tight_layout()
plt.show()

تعداد آگهی‌های منتشر شده در ماه‌های مختلف را برای فروش و اجاره بررسی کنید. آیا تعداد آگهی‌های فروش و اجاره در زمان‌های مشخصی از سال افزایش چشم‌گیری داشته است؟

In [ ]:
timestmap.value_counts().plot.bar()


بر روی نقشه‌ی جغرافیایی heatmap آگهی‌های هر منطقه را رسم کنید. تراکم آگهی‌ها کدام منطقه بیشتر است؟


In [ ]:
location_info = Divar_data[['location_latitude','location_longitude','location_radius']]
location_info.dropna(inplace=True)
location_info = location_info[['location_latitude','location_longitude']].value_counts()


tehran_map = folium.Map(location =[35.6892, 51.3890],zoom_start=13 )
loc = location_info.index.to_frame()
locc = pd.DataFrame({'location_latitude':loc["location_latitude"],'location_longitude':loc["location_longitude"],'count':location_info})
tehran_map = folium.Map(location =[35.6892, 51.3890],zoom_start=13 )
HeatMap(data=locc, radius=15, max_zoom=13).add_to(tehran_map)
tehran_map

توزیع قیمت فروش‌(price_value) را برای دسته‌بندی‌های سطح سه در یک نمودار رسم کنید.

In [ ]:
ejare_mah = pd.DataFrame({"Category":Divar_data['cat3_slug'],"Price":Divar_data['price_value']})




plt.figure(figsize=(20, 6))
sns.boxplot(x='Category', y='Price', data=ejare_mah, palette='Set2')


plt.title('tojie qheymat dastebandi_3', fontsize=14)
plt.xlabel('dastebandi_3', fontsize=12)
plt.ylabel('qhymat', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=1)


plt.tight_layout()
plt.show()




ترند میانگین قیمت اجاره بر حسب ماه‌های قرار گرفتن آگهی‌ها رسم کنید.(دقت کنید که ماه‌ها باید به تاریخ شمسی و خوانا باشند.)

In [ ]:
def to_shamsi(x):
    date = jdatetime.fromgregorian(datetime = x)
    date = (2022,date.month,1)
    return pd.Timestamp(year=date[0],month=date[1],day=date[2])

In [ ]:
#month_of_set = pd.DatetimeIndex(pd.to_datetime(pd.to_datetime(Divar_data['created_at_month']).map(lambda x : str(jdatetime.fromgregorian(datetime = x)))))
#month_of_set = pd.Series(data = Divar_data['rent_value'].copy().values , index = month_of_set)
pd_datetime = pd.Series(pd.to_datetime(Divar_data['created_at_month']).map(lambda x : to_shamsi(x)))


In [ ]:
pd_datetime

In [ ]:
month_of_set = pd.DatetimeIndex(pd.to_datetime(pd_datetime))
month_of_set = pd.Series(data = Divar_data['rent_value'].values , index = month_of_set)
month_of_set

month_of_set = month_of_set.groupby(pd.Grouper(freq = "M")).mean()
dpp = pd.DataFrame({'index' :timestamptomonth(month_of_set.index),'mean' : month_of_set.values  })
perisan_month=('Farvardin','Ordibehesht','Khordad','Tir','Mordard','Sharivar','Mehr','Aban','Azar','Day','Bahman','Esfand')
plt.figure(figsize=(12, 6))
plt.plot(perisan_month, dpp['mean'], marker='o', linestyle='-')
plt.xlabel('month')
plt.ylabel('miyangine ejare')
plt.title('trend miyangin ejare')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

در طول زمان قیمت‌های اسمی افزایش پیدا می‌کنند اما این افزایش لزوما به معنی بالارفتن ارزش واقعی ملک نیست و می‌تواند ناشی از تورم باشد. به ازای میانگین مبلغ قیمت(price_value) در سال‌های ۱۴۰۰ تا ۱۴۰۳ قیمت حقیقی را محاسبه کنید و بررسی کنید ترند قیمت حقیقی چگونه است.

In [ ]:
def toshamsi(x):
    date = jdatetime.fromgregorian(datetime = x)
    return date.year

date = pd.Series(pd.to_datetime(Divar_data['created_at_month']))
date = date.map(lambda x : toshamsi(x))


In [ ]:
date = pd.DataFrame({'Years': date.values , 'price_value' : Divar_data['price_value'].values })


In [ ]:
date.sort_values(by = ['Years'])
groped = date.groupby(pd.Grouper(key = 'Years')).mean()
groped = groped[0:4]
groped.sort_index(inplace = True)
true_values = [(groped.values[0]*100)/150 ,(groped.values[0]*100)/240,(groped.values[0]*100)/408,(groped.values[0]*100)/571]


plt.figure(figsize=(10, 6))
plt.plot(groped.index, groped.values, label= 'Market_Value', marker='o')
plt.plot(groped.index, true_values, label='True_Value', marker='s')
plt.xlabel('Year')
plt.ylabel('Value')
plt.title('copmarision between years 1400-1403')
plt.legend()
plt.grid(True)
plt.show()

 ماتریس هم‌بستگی را برای مبلغ قیمت، متراژ زمین، زیربنا، ظرفیت نفرات، تعداد اتاق‌ها و طول و عرض جغرافیایی رسم نمایید.

In [ ]:
matrix_data = pd.DataFrame({
    'price_value' : Divar_data['price_value'].values ,
    'land_size' : Divar_data['land_size'].values ,
    'building_size' : Divar_data['building_size'].values ,
    'person_capacity' : Divar_data['regular_person_capacity'].values,
    'rooms_count' : Divar_data['rooms_count'].values ,
    'location_longitude' : Divar_data['location_longitude'].values,
    'location_latitude' : Divar_data['location_latitude'].values
})


for i in Divar_data['rooms_count'].index:
    if   matrix_data['rooms_count'][i] == 'یک' : matrix_data['rooms_count'][i] = 1
    elif matrix_data['rooms_count'][i] == 'دو' : matrix_data['rooms_count'][i] = 2
    elif matrix_data['rooms_count'][i] == 'سه' : matrix_data['rooms_count'][i] = 3
    elif matrix_data['rooms_count'][i] == 'چهار' : matrix_data['rooms_count'][i] = 4
    elif matrix_data['rooms_count'][i] == 'بدون اتاق': matrix_data['rooms_count'][i] = 0
    elif matrix_data['rooms_count'][i] =='پنج یا بیشتر' : matrix_data['rooms_count'][i] = 5

In [ ]:
correlation_matrix = matrix_data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('matrix_hambastegi')
plt.show()

می‌خواهیم بررسی کنیم خانه‌هایی که دارای بالکن، آسانسور، نگهبان، باربیکیو و استخر هستند عمدتا در کدام مناطق قرار دارند. با نمودار مناسب این موضوع را نشان دهید.

In [ ]:
last_data = Divar_data[['has_security_guard','has_elevator','has_pool','has_barbecue','has_balcony','location_longitude','location_latitude']]
last_data.dropna(subset=['location_longitude','location_latitude'],inplace = True)
map = folium.Map(location=[35.7, 51.4], zoom_start=12)

for emcan in ['has_balcony', 'has_elevator', 'has_security_guard', 'has_barbecue', 'has_pool']:
    filtered_last_data = last_data[last_data[emcan] == True]
    heat_data = [[row['location_latitude'], row['location_longitude']] for _, row in last_data.iterrows()]
    HeatMap(heat_data, name=f"{emcan} ({len(filtered_last_data)})", radius=12, blur=10, max_zoom=15).add_to(map)

folium.LayerControl().add_to(map)

map

با توجه به رشد مهاجرت افراد از شهر‌های کوچکتر به کلان‌شهر‌ها و تراکم جمعیت در این نواحی، تصور می‌شود که میانگین مساحت خانه‌های مسکونی در کلان‌شهرها نسبت به شهر‌های کوچک و روستاها کمتر است. آیا مجموعه داده‌ این فرضیه را پشتیبانی می‌کند؟(برای دسته‌بندی شهرها به کلان‌شهر و شهر کوچک می‌توانید از این مجموعه داده استفاده کنید.)

این فرض درستی است .اما این داده ها این فرض را تایید نمی کند.زیرا صرفا مساحت خانه های اگهی این فرض را تایید نمی کند
باید مساحت خانه ها را و تراکم جعیت را درطول زمان برسی کرد.




معمولا این جمله را می‌شنویم که «قدیما خونه‌ها دلبازتر بود!» برای بررسی این فرضیه، آیا میانگین مساحت خانه‌های قدیمی‌ساخت نسبت به خانه‌های جدید ساخت بیشتر است؟( خانه‌های قدیمی‌ساخت را خانه‌هایی در نظر بگیرید که قبل از سال ۹۶ ساخته شده‌اند.)

طبق این داده ها خیر

In [ ]:
year_size = pd.DataFrame({"year" : pd.to_datetime(Divar_data['created_at_month']).map(lambda x : x.year),"size":Divar_data['building_size']})
year_size.sort_values(by=['year'],inplace =True)
year_size_group = year_size.groupby(pd.Grouper(key = 'year')).mean()
year_size_group

plt.figure(figsize=(10, 6))
plt.plot(year_size_group.index,year_size_group.values, label= 'Size or year', marker='o')
plt.xlabel('Year')
plt.ylabel('size')
plt.grid = True

plt.show()

داشتن سند تجاری (یا هر نوع سند ملکی) در املاک به این معنی است که سند مالکیت معتبر، رسمی و قانونی برای ملک تجاری دارید. این سند نشان می‌دهد که شما صاحب قانونی ملک تجاری هستید و می‌توانید از حقوق مالکیت آن استفاده کنید. بررسی کنید که آیا داشتن سند تجاری(has_business_deed) بر میانگین قیمت فروش ملک تجاری تاثیر معناداری دارد؟

In [ ]:
sanad_price =  pd.DataFrame({"sanad" : Divar_data['has_business_deed'],"price":Divar_data['price_value']})
sanad_price_group =sanad_price.groupby(pd.Grouper(key = 'sanad')).mean()
sanad_price_group
